# DicoCo

*Marie Margueritat*

# Librairie

In [1]:
# import necessaire à la création des widgets
import ipywidgets as widgets
import pandas
import numpy

from bs4 import BeautifulSoup
from IPython import display as audioD
import IPython.display as ipd
from requests_html import HTMLSession
import math
from IPython.display import display, clear_output, HTML, Markdown 
from pylab import specgram, show
from moviepy.editor import AudioFileClip

pandas.options.mode.chained_assignment = None 
session = HTMLSession()

# Les données

#### On récupère le DF

In [2]:
dfDicoBuch = pandas.read_csv('./Data/dfDicoBuch.csv')
# On conserve les données qui nous sont utiles
dfDicoBuch = dfDicoBuch[["motVedette","nbSyll","pron","pronSampa","ipa"]]
dfDicoBuch = dfDicoBuch.replace(numpy.nan, '', regex=True)

#### On scrappe toute la table en créant 2 colonnes (une lienUK et une lien US)

In [3]:
def scrapping(dfEtudie):
    
    url = "https://www.ldoceonline.com/dictionary/"
    listeLienUK = []
    listeLienUS = []
    
    for mot in dfEtudie["motVedette"] :

        # On créé le lien URL
        urlComplet = url + mot
        response = session.get(urlComplet)
        html_doc = response.content

        # Parsing
        soup = BeautifulSoup(response.text,'html.parser')
        souplettesBody = soup.find('body')
        finded = souplettesBody.findAll('div',{"class":"dictionary"})

        # Lien UK
        for v in finded:
            baliseAnglais = v.findAll('span',{"class":"speaker brefile fas fa-volume-up hideOnAmp"})
        for f in baliseAnglais:
            liensAnglais = f['data-src-mp3']
        lienAnglais = liensAnglais.replace(liensAnglais[-15:],"")
        listeLienUK.append(lienAnglais)

        # Lien US 
        for v in finded:
            baliseUS = v.findAll('span',{"class":"speaker amefile fas fa-volume-up hideOnAmp"})
        for f in baliseUS:
            liensUS = f['data-src-mp3']
        lienUS = liensUS.replace(liensUS[-15:],"")
        listeLienUS.append(lienUS)
        
        
    dfEtudie["lienUK"] = pandas.Series(listeLienUK).values
    dfEtudie["lienUS"] = pandas.Series(listeLienUS).values
    
    
    return dfEtudie


# Les widgets

### Ligne 1

In [4]:
filtreHWD = widgets.Text(value="",description='filtre hwd',layout={'width': '200px'} )

In [5]:
filtreIPA = widgets.Text(value="",description='filtre ipa',layout={'width': '200px'} )

In [6]:
occMax = widgets.IntSlider(min=0,max=10,step=1,description='Occ.Max:',value = 5)

In [7]:
bouton = widgets.Button(description='Démarrer', icon = "check")

### Ligne 2

Les variables

In [8]:
# Widgets
dataFrame = widgets.Output()
audio = widgets.Output()

In [9]:
# Création des spectrometres 
def creaSpectro(lienUK, lienUS):
    # création des spectromètres
    audio = AudioFileClip(lienUK) 
    sound_array = audio.to_soundarray()
    specgram(sound_array[:,0])
    show()
    audio = AudioFileClip(lienUS) 
    sound_array = audio.to_soundarray()
    specgram(sound_array[:,0])
    show()

In [10]:
# Création des audio
def creaAudio(dfEtudie):
    clear_output()
    listeWid = widgets.Output()
    lU = widgets.Output()
    sU = widgets.Output()

    for numLigne in range(len(dfEtudie)):
        
        
        labelUK = widgets.Label("Brit")
        itemsUK = audioD.Audio(url = dfEtudie["lienUK"][numLigne])
        spectroUK = dfEtudie["lienUK"][numLigne]
    
        labelUS = widgets.Label("Amer")
        itemsUS = audioD.Audio(url = dfEtudie["lienUS"][numLigne])
        audioUK = widgets.Output()
        audioUK.append_display_data(itemsUK)
        audioUS = widgets.Output()
        audioUS.append_display_data(itemsUS)
        spectroUS = dfEtudie["lienUS"][numLigne]
        
        lU = widgets.HBox([labelUK,audioUK,labelUS,audioUS])

        # Les mots

        labelMot = widgets.Label(dfEtudie["motVedette"][numLigne])
        labelPron = widgets.Label(str(dfEtudie["pron"][numLigne]))
        label = widgets.HBox([labelMot, labelPron])


        listeWid.append_display_data(widgets.VBox([label,lU,sU]))
        creaSpectro(spectroUK, spectroUS)
                        
    return listeWid

In [11]:
def on_button_clicked(_):
    nbLigne = occMax.value
    HWD = filtreHWD.value 
    IPA = filtreIPA.value
    dfEtudie = ""

    with dataFrame:
        clear_output()
        # Filtre en fontion des variables précédentes 
        if (HWD != ""):
            dfEtudie = dfDicoBuch[(dfDicoBuch.motVedette.str.match(HWD))]
            if(IPA != ""):
                dfEtudie = dfEtudie[(dfEtudie.pron.str.match(IPA))]
        elif (IPA != ""):
            dfEtudie = dfDicoBuch[(dfDicoBuch.pron.str.match(IPA))]

        if(len(dfEtudie) > nbLigne):
            dfEtudie = dfEtudie.sample(n=nbLigne).copy()
        dfEtudie = dfEtudie.reset_index()    
        # On affiche
        display(dfEtudie)
        
    with audio :
        clear_output()
        dfEtudie = scrapping(dfEtudie)  
        
        listeAudio = creaAudio(dfEtudie) 
        display(listeAudio)
        
            
bouton.on_click(on_button_clicked)

## Layout 

In [12]:
ligne1 = widgets.HBox([filtreHWD, filtreIPA, occMax, bouton])
ligne2 = widgets.HBox([dataFrame])
ligne3 = widgets.HBox([audio])

tout = widgets.VBox([ligne1, ligne2, ligne3])

In [13]:
tout